In [ ]:
# Instalando o upgrade do pip
# Instalando a biblioteca pymssql

!pip install --upgrade pip
!pip install --upgrade pandas

In [ ]:
# Importando Pandas, pymssql e adal

import pandas as pd

In [ ]:
# Buscando os dados da API e salvando em CSV

api = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial=%2701-01-2019%27&@dataFinalCotacao=%2712-31-2025%27&$top=9000&$format=text/csv&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao"
csv = pd.read_csv(api)
csv.head()

In [ ]:
# Importando Spark e Functions

from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
# Criando DataFrame

df = spark.createDataFrame(csv)
display(df)
df.dtypes

Acessando o Azure SQL

In [ ]:
!pip install msal

In [ ]:
import msal

In [ ]:
#Id do locatário/locatário: Localizado em Registros de Aplicativos do Azure Active Directory -> app-user-databricks
tenant_id = 'b4920c82-7581-491a-9dab-cd2ade2f3ebd'

#Autoridade:
msal_authority = 'https://login.windows.net/' + tenant_id
   
#url do recurso database
resource_url = 'https://database.windows.net/.default'
   
# Service Principal Client ID - Criado em secrets no Key Vault
service_principal_id = dbutils.secrets.get(scope = 'scope-adb-estudos', key = 'app-reg-adb')
       
# Service Principal Secret - Criado em secrets no Key Vault
service_principal_secret = dbutils.secrets.get(scope = 'scope-adb-estudos', key = 'app-user-databricks')

In [ ]:
# Variaveis e string de conexão ao Azure SQL Server
jdbcHostname  = 'sql-estudo.database.windows.net'
jdbcDatabase  = 'db-estudos'
jdbcPort      = 1433
jdbcUrl       = f'jdbc:sqlserver://{jdbcHostname}:{jdbcPort};database={jdbcDatabase}'

In [ ]:
#Criando a instância que sera usada durante todo o ciclo de vida do aplicativo.
instance = msal.ConfidentialClientApplication(service_principal_id, service_principal_secret, msal_authority)
   
#Enviando solicitação ao AAD para obter um token.
token = instance.acquire_token_for_client(resource_url)
   
#acesando o token
access_token = token['access_token']

Inseridos os dados na tabela Stage

In [ ]:
# Escrevendo os dados na tabela stage

table = 'dolar_marcus_araujo.dolar_stage_marcus_araujo02'

df.write\
    .format('jdbc')\
    .mode('overwrite')\
    .option('url', jdbcUrl)\
    .option('databaseName', jdbcDatabase)\
    .option('dbtable', table)\
    .option('accesstoken', access_token)\
    .save()

In [ ]:
#Leitura dos dados na tabela stage

df_read = spark.read\
            .format('jdbc')\
            .option('url', jdbcUrl)\
            .option('databaseName', jdbcDatabase)\
            .option('dbtable', table)\
            .option('accesstoken', access_token)\
            .load()

display(df_read)